<a href="https://colab.research.google.com/github/madhurendra0089/Assignment/blob/main/2D_CNN_MRI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

In [ ]:
# Set the path to the preprocessed data
path = "/content/drive/MyDrive/MRI_ED/"

# Load the preprocessed data
x_train = np.load(path + 'x_train.npy')
x_test = np.load(path + 'x_test.npy')
x_val = np.load(path + 'x_val.npy')

y_train = np.load(path + 'y_train.npy')
y_test = np.load(path + 'y_test.npy')
y_val = np.load(path + 'y_val.npy')

print(f"#x_train: {len(x_train)} #y_train: {len(y_train)} #x_val: {len(x_val)} #y_val: {len(y_val)} #x_test: {len(x_test)} #y_test: {len(y_test)}")

# Flatten 3D data into 2D slices
def flatten_3d_to_2d(data):
    temp = []
    for i in range(len(data)):
        for j in range(data.shape[-1]):  # Iterate over the third dimension (slices)
            temp.append(data[i, :, :, j])
    return np.array(temp)

x_train = flatten_3d_to_2d(x_train)
x_test = flatten_3d_to_2d(x_test)
x_val = flatten_3d_to_2d(x_val)

print("New shapes:")
print(f"x_train shape: {x_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"x_val shape: {x_val.shape}")

# Expand dimensions for CNN compatibility
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)
x_val = np.expand_dims(x_val, axis=-1)

print("After expanding dimensions:")
print(f"x_train shape: {x_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"x_val shape: {x_val.shape}")

# Adjust `y_train`, `y_test`, `y_val` to align with the flattened data
y_train = np.repeat(y_train, x_train.shape[0] // len(y_train))
y_test = np.repeat(y_test, x_test.shape[0] // len(y_test))
y_val = np.repeat(y_val, x_val.shape[0] // len(y_val))

print("Adjusted labels:")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")
print(f"y_val shape: {y_val.shape}")


In [2]:
# Define the CNN model
def CNN(width=100, height=100):
    inputs = Input((width, height, 1))
    x = Conv2D(4, 3, activation='relu', padding='same')(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(8, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(16, 3, activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(32, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(64, 3, activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(32, activation='relu')(x)
    x = Dense(16, activation='relu')(x)
    outputs = Dense(1, activation='sigmoid')(x)
    return Model(inputs, outputs)

model = CNN(width=x_train.shape[1], height=x_train.shape[2])
model.summary()

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    x_train, y_train,
    batch_size=16,
    epochs=100,
    validation_data=(x_val, y_val)
)

In [ ]:
# Evaluate the model's performance
y_pred = model.predict(x_test)
y_pred_binary = (y_pred > 0.5).astype(int)

def performance(y_test, y_pred_binary):
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred_binary))
    print("Accuracy:", accuracy_score(y_test, y_pred_binary))
    print("Precision:", precision_score(y_test, y_pred_binary))
    print("Recall:", recall_score(y_test, y_pred_binary))
    print("F1 Score:", f1_score(y_test, y_pred_binary))

performance(y_test, y_pred_binary)